# `Utf8JsonWriter`

In [1]:
using System.IO;
using System.Text.Json;

public static Utf8JsonWriter WriteObject(this Utf8JsonWriter writer, Action writerAction)
{
    if (writer == null) return writer;

    writer.WriteStartObject();
    writerAction?.Invoke();
    writer.WriteEndObject();

    return writer;
}

public static Utf8JsonWriter WriteObject(this Utf8JsonWriter writer, string propertyName, Action writerAction)
{
    if (writer == null) return writer;
    if (string.IsNullOrEmpty(propertyName)) throw new ArgumentNullException(nameof(propertyName));

    writer.WritePropertyName(propertyName);
    writer.WriteStartObject();
    writerAction?.Invoke();
    writer.WriteEndObject();

    return writer;
}

In [2]:

static string getElementJson()
{
    var matchPropertyName = "myIndexNestedObject.myProperty";
    var matchPropertyExpectedValue = 99;

    using var stream = new MemoryStream();
    using var writer = new Utf8JsonWriter(stream);

    writer.WriteObject(() =>
    {
        writer.WriteNumber(matchPropertyName, matchPropertyExpectedValue);
    });

    writer.Flush();

    string json = Encoding.UTF8.GetString(stream.ToArray());

    return json;
}

In [3]:
var stream = new MemoryStream();
var writer = new Utf8JsonWriter(stream);

writer.WriteStartObject();
writer.WriteEndObject();
writer.Flush();

string json = Encoding.UTF8.GetString(stream.ToArray());

json

{}

In [4]:
const string match = "match";
const string path = "path";
const string nested = "nested";
const string query = "query";

var options = new JsonWriterOptions
{
    Indented = true,
};

var pathPropertyExpectedValue = "myIndexNestedObject";

var matchPropertyName = "myIndexNestedObject.myProperty";
var matchPropertyExpectedValue = 99;

var stream = new MemoryStream();
var writer = new Utf8JsonWriter(stream, options);

writer.WriteObject(() =>
{
    writer.WriteObject(query, () =>
    {
        writer.WriteObject(nested, () =>
        {
            writer.WriteString(path, pathPropertyExpectedValue);

            writer.WriteObject(query, () =>
            {
                writer.WriteObject(match, () =>
                {
                    writer.WriteNumber(matchPropertyName, matchPropertyExpectedValue);
                });
            });
        });
    });
});

writer.Flush();

string json = Encoding.UTF8.GetString(stream.ToArray());

json

{
  "query": {
    "nested": {
      "path": "myIndexNestedObject",
      "query": {
        "match": {
          "myIndexNestedObject.myProperty": 99
        }
      }
    }
  }
}

In [5]:
var jDocument = JsonDocument.Parse(json);

var pathProperty = jDocument.RootElement
    .GetProperty(query)
    .GetProperty(nested)
    .GetProperty(path);

$"{pathProperty.GetString()}, {pathPropertyExpectedValue}"

myIndexNestedObject, myIndexNestedObject

In [6]:
var matchProperty = jDocument.RootElement
    .GetProperty(query)
    .GetProperty(nested)
    .GetProperty(query)
    .GetProperty(match)
    .GetProperty(matchPropertyName);

$"{matchProperty.GetInt32()}, {matchPropertyExpectedValue}"

99, 99

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼
